# Q&A chain and Agent over SQL Database

## Architecture
At a high level, the steps of any SQL chain and agent are:
- Convert question to SQL query using LLM model.
- Execute the SQL query
- Getting the model to respond to user input using the query results.

In [1]:
import os
from dotenv import load_dotenv
import openai
from langchain_openai import ChatOpenAI

In [2]:
# Setup model
load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
openai.api_key = api_key

In [3]:
llm = ChatOpenAI(temperature=0)

In [4]:
from langchain.utilities import SQLDatabase
from langchain_experimental.sql import SQLDatabaseChain
from langchain.prompts import PromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
from langchain.chains import create_sql_query_chain

In [5]:
host = "193.206.183.37"
port = "3306"
username = "root"
password = "root"
database_schema = "analytics"
mysql_uri = f"mysql+pymysql://{username}:{password}@{host}:{port}/{database_schema}"
#print(mysql_uri)
db = SQLDatabase.from_uri(mysql_uri,sample_rows_in_table_info = 5)
chain = create_sql_query_chain(llm,db)
#print(chain)

In [6]:
print(db.dialect)
print(db.get_usable_table_names())

mysql
['customer', 'sale']


In [7]:
db.run("Select count(*) from customer LIMIT 10;")

'[(1001,)]'

In [8]:
response = chain.invoke({"question":"How many customer are there?"})
response

'SELECT COUNT(`customerid`) AS total_customers FROM customer;'

In [9]:
db.run(response)

'[(1001,)]'

In [10]:
response = chain.invoke({"question":"How many customers from bronx?"})
response

"SELECT COUNT(*) AS total_customers\nFROM customer\nWHERE city = 'Bronx';"

In [11]:
db.run(response)

'[(2,)]'

In [15]:
response = chain.invoke({"question":"How many customers from bronx and list their name and phone?"})
print(response)
db.run(response)

SELECT `firstname`, `lastname`, `phone`
FROM `customer`
WHERE `city` = 'Bronx'
LIMIT 5;


"[('Cecile', 'Clergeau', '(718) 405-2868'), ('Guoqing', 'Chen', '(718) 829-2436')]"

In [13]:
response = chain.invoke({"question":"Which state are customers most from?"})
print(response)
db.run(response)

SELECT `state`, COUNT(`state`) AS state_count
FROM `customer`
GROUP BY `state`
ORDER BY state_count DESC
LIMIT 1;


"[('NY', 79)]"

In [14]:
response = chain.invoke({"question":"Give me the top 5 customers who paid the highest shipping charges"})
print(response)
db.run(response)

SELECT c.customerid, c.firstname, c.lastname, c.phone, s.shipping
FROM customer c
JOIN sale s ON c.customerid = s.customerid
ORDER BY s.shipping DESC
LIMIT 5;


"[('customerid', 'firstname', 'lastname', 'phone', 'shipping'), ('3286', 'Cyrus', 'Gallagher', '(408) 626-8013', '9.95'), ('1376', 'Richard', 'Magner', '(205) 921-9237', '9.95'), ('1827', 'Kirill', 'Yurov', '(718) 434-6188', '9.95'), ('2609', 'Rajiv', 'Banker', '(812) 546-4257', '9.95')]"